✅ Parte 1: Preparação do Ambiente e dos Dados
O que está acontecendo:
Instalamos as dependências.

Carregamos o dataset JSON.

Preparamos os dados no formato de instrução-resposta (Instruction Tuning), onde:

Instrução: "Pergunta sobre título X: Y?"

Resposta: "Descrição do produto."

1.1 Fonte dos Dados
Os dados utilizados para o fine-tuning foram coletados do trn.json.

O dataset contém exemplos no formato: título do produto, pergunta relacionada e resposta descritiva.

1.2 Limpeza e Formatação
Foram selecionados os primeiros registros para treino.

Cada registro foi formatado no formato de instrução e resposta.

Dados foram convertidos para formato Dataset do Hugging Face para facilitar a tokenização e o treinamento.

1.3 Tokenização e Pré-processamento
Foi usada a tokenização da biblioteca Transformers com truncagem para limitar o tamanho máximo do input.

1.4 Carregamento dos dados de Teste
Carregamento do conjunto de teste (tst.json) e o conjunto de rótulos verdadeiros (lbl.json) para validação da performance do modelo, usados para avaliação pós-treinamento.

O dataset tokenizado foi preparado para treinamento causal language modeling.



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Instalar dependências
!pip install -q transformers datasets peft bitsandbytes accelerate

In [3]:
import json

data = []
with open('/content/drive/MyDrive/Colab Notebooks/Datasets/trn.json') as f:
    for i, line in enumerate(f):
        data.append(json.loads(line))
        if i >= 9:  # pega só os 10 primeiros para inspeção
            break

print("EXEMPLO TRN JSON:", data)

data2 = []
with open('/content/drive/MyDrive/Colab Notebooks/Datasets/lbl.json') as f1:
    for i, line in enumerate(f1):
        data2.append(json.loads(line))
        if i >= 9:  # pega só os 10 primeiros para inspeção
            break

print("EXEMPLO LBL JSON:", data2)

data3 = []
with open('/content/drive/MyDrive/Colab Notebooks/Datasets/tst.json') as f3:
    for i, line in enumerate(f3):
        data2.append(json.loads(line))
        if i >= 9:  # pega só os 10 primeiros para inspeção
            break

print("EXEMPLO TST JSON:", data2)

EXEMPLO TRN JSON: [{'uid': '0000031909', 'title': 'Girls Ballet Tutu Neon Pink', 'content': 'High quality 3 layer ballet tutu. 12 inches in length', 'target_ind': [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 111], 'target_rel': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}, {'uid': '0000032034', 'title': 'Adult Ballet Tutu Yellow', 'content': '', 'target_ind': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 16, 33, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71], 'target_rel': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}, {'uid': '0000913154', 'title': 'The Way Things Work: An Illustr

✅ trn.json → conjunto de treinamento

Usado para fazer o fine-tuning do modelo.

Já tem os campos title e content preenchidos.

Serve para o modelo aprender a gerar respostas ou descrições com base nos títulos.

✅ tst.json → conjunto de teste / validação

Possui content preenchido.

Serve como "gabarito" para avaliar a qualidade do modelo após o fine-tuning.

✅ lbl.json → rótulos previstos

Tem estrutura igual ao tst.json, mas content vazio.

É onde o modelo irá preencher suas respostas geradas para posterior comparação com o tst.json.

In [5]:


# Importar bibliotecas
import json
import pandas as pd
from datasets import Dataset

data = []
#Documentos estão também no github
with open('/content/drive/MyDrive/Colab Notebooks/Datasets/trn.json') as f:
    for i, line in enumerate(f):
        data.append(json.loads(line))
        if i >= 9:  # pega só os 10 primeiros para inspeção
            break

print(data)

# Converter em DataFrame
df = pd.DataFrame(data)

# Visualizar primeiras linhas
print(df.head())

# Preparar dados no formato de instrução
def prepare_instruction(row):
    return {
        'instruction': f"Pergunta sobre o título '{row['title']}': Descreva o produto.",
        'response': row['content']
    }

prepared_data = df.apply(prepare_instruction, axis=1).tolist()

# Converter para Hugging Face Dataset
dataset = Dataset.from_list(prepared_data)

# Ver exemplo
print(dataset[0])


[{'uid': '0000031909', 'title': 'Girls Ballet Tutu Neon Pink', 'content': 'High quality 3 layer ballet tutu. 12 inches in length', 'target_ind': [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 111], 'target_rel': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}, {'uid': '0000032034', 'title': 'Adult Ballet Tutu Yellow', 'content': '', 'target_ind': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 16, 33, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71], 'target_rel': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}, {'uid': '0000913154', 'title': 'The Way Things Work: An Illustrated Encyclopedia 

✅ Parte 2: Carregamento do Modelo Base e Avaliação Antes do Fine-Tuning
O que está acontecendo:
Carregamos um modelo base TinyLlama/TinyLlama-1.1B-Chat-v1.0

Testamos como o modelo responde antes do fine-tuning.

Isso cria um baseline de comparação.

1️⃣ Instalar bibliotecas necessárias

In [4]:
!pip install transformers
!pip install bitsandbytes
!pip install accelerate
!pip install sentencepiece


In [6]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForCausalLM.from_pretrained(model_name)

generator = pipeline("text-generation", model=base_model, tokenizer=tokenizer)

prompt = "Question: What is Girls Ballet Tutu Neon?\nAnswer:"

result = generator(prompt, max_length=100, do_sample=True, temperature=0.7)
generated_text = result[0]['generated_text']
generated_only = generated_text[len(prompt):]

print("Generated Answer By TinyLlama Without Fine-Tuning Training:")
print(generated_only)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Generated Answer By TinyLlama Without Fine-Tuning Training:
 Girls Ballet Tutu Neon is a new product line for Ballerina. It features a neon color and is made of high quality, stretchy fabric. The tutu is available in sizes 3-11 and the color is available in two different styles. It is perfect for all your ballet needs. You can purchase it online or at your nearest Ballerina store.


E se fosse uma tarefa diferente?
Se fosse QA com respostas extraídas diretamente de um texto fixo (e não geração livre), talvez usaria AutoModelForQuestionAnswering, que é para reading comprehension, onde o modelo seleciona spans no texto, e não gera texto livre.

Mas como o objetivo é geração de respostas contextualizadas, o causal LM é o caminho.

✅ Parte 3: Fine-tuning com PEFT (LoRA)
O que está acontecendo:
Realizamos fine-tuning eficiente com LoRA, que evita treinar todos os parâmetros do modelo.

Criamos um Trainer e fazemos o ajuste.

Usamos poucos epochs, pois a ideia é apenas ilustrar.

ETAPAS ✈:

* Configura o LoRA	Define como o modelo vai ser adaptado (leve e eficiente)

* Aplica LoRA no modelo	Insere as camadas LoRA no modelo base

* Tokeniza os dados	Transforma dados (instruction + response) em tokens para o modelo

* Prepara batches	Usa o DataCollator para preparar os batches com padding e labels

* Define argumentos de treino	Como batch size, epochs, onde salvar etc.

* Cria Trainer	Junta tudo e gerencia o ciclo de treinamento
Executa treinamento	Ajusta os parâmetros LoRA com base nos dados


In [7]:
from peft import get_peft_model, LoraConfig, TaskType
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
import os
os.environ["WANDB_DISABLED"] = "true"

# Configuração LoRA
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1
)

# Aplicar PEFT no modelo
model = get_peft_model(base_model, peft_config)

# Tokenização
def tokenize_function(examples):
    combined = [
        instr + "\n" + resp
        for instr, resp in zip(examples['instruction'], examples['response'])
    ]
    return tokenizer(combined, truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Data Collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Argumentos de treinamento
training_args = TrainingArguments(
    output_dir="./TrainedModelTechChallenge3",
    per_device_train_batch_size=1,
    num_train_epochs=5,  # Aumente para resultados reais
    save_steps=10,
    logging_steps=10
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Iniciar Fine-tuning
trainer.train()


/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-7-f806e640181a>:41: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,3.474800
20,3.308000
30,3.150600
40,3.028800
50,2.961800


TrainOutput(global_step=50, training_loss=3.184817199707031, metrics={'train_runtime': 10.0831, 'train_samples_per_second': 4.959, 'train_steps_per_second': 4.959, 'total_flos': 15503512596480.0, 'train_loss': 3.184817199707031, 'epoch': 5.0})

3.1 Modelo Base
Utilizado o modelo pré-treinado TinyLlama-1.1B-Chat-v1.0 como foundation model.

Classe utilizada: AutoModelForCausalLM para geração de texto autoregressiva.

3.2 Técnica de Fine-Tuning
Aplicada técnica de LoRA (Low-Rank Adaptation) para tornar o fine-tuning eficiente e leve.

Configurações do LoRA:

r=8

lora_alpha=16

lora_dropout=0.1

Permitiu atualização rápida de poucos parâmetros, preservando o conhecimento do modelo base.

2.3 Parâmetros de Treinamento
Batch size: 1 (devido a limitações de memória)

Número de épocas: 1 (para demonstração; recomendado aumentar para produção)

Learning rate padrão do Trainer.

Salvamento do checkpoint a cada 10 steps.

Uso do DataCollatorForLanguageModeling com mlm=False para treinamento causal.

2.4 Execução do Treinamento
Treinamento executado em ambiente Google Colab com GPU.

✅ Esse resultado está bom?
Sim, considerando que o objetivo era testar e validar o pipeline de fine-tuning.

Para um fine-tuning real, normalmente usamos:

Mais épocas (num_train_epochs=3~10).

Ajuste de learning_rate.

Batch maior, se a infra permitir.

Avaliação em um conjunto de validação.

✅ Parte 4: Avaliação do Modelo Após o Fine-Tuning
O que está acontecendo:
Reutilizamos a pipeline.

Geramos resposta para o mesmo exemplo anterior.

Compararmos antes e depois do fine-tuning.
Subimos os arquivos tst.json e o lbl.json para a comparação e para gerar métricas do modelo.

tst.json → conjunto de teste / validação

Possui content preenchido.

Serve como "gabarito" para avaliar a qualidade do modelo após o fine-tuning.

lbl.json → rótulos previstos

Tem estrutura igual ao tst.json, mas content vazio.

É onde o modelo irá preencher suas respostas geradas para posterior comparação com o tst.json



✅ O que o código faz:
Carrega os dados (tst.json com respostas corretas e lbl.json com os prompts).

Seleciona um subconjunto pequeno para teste.

Gera respostas automáticas com um modelo de linguagem usando model.generate().

Salva as respostas geradas em lbl_generated.json.

Avalia as respostas com duas métricas:

BLEU: mede a precisão de n-gramas (semelhança palavra a palavra).

ROUGE: mede a sobreposição de sequências, focando mais no recall.

✅ Principais funções:
model.generate(): cria o texto.

load_metric(): carrega métricas BLEU e ROUGE.

bleu.compute(), rouge.compute(): calculam as pontuações.

✅ Pra que serve?
Verificar se o modelo está gerando respostas similares às corretas, de forma automática e objetiva.

In [2]:
!pip install rouge_score


In [8]:
# --- Importações ---
import torch
from tqdm import tqdm
import json
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
from datasets import load_metric

# --- Carregar modelo base e tokenizer ---
base_model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # ajuste para o modelo correto
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
base_model = AutoModelForCausalLM.from_pretrained(base_model_name)

# --- Carregar modelo fine-tunado com LoRA ---
lora_model_path = "/content/ModeloTreinadoTechChallenge3/checkpoint-50"  # ajuste conforme seu caminho
model = PeftModel.from_pretrained(base_model, lora_model_path)
model.eval()

# --- Ajuste do device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# --- Carregar tst.json e lbl.json ---
tst_data = []
with open('/content/drive/MyDrive/Colab Notebooks/Datasets/tst.json') as f:
    for line in f:
        tst_data.append(json.loads(line))

lbl_data = []
with open('/content/drive/MyDrive/Colab Notebooks/Datasets/lbl.json') as f:
    for line in f:
        lbl_data.append(json.loads(line))

print(f"TST samples: {len(tst_data)}, LBL samples: {len(lbl_data)}")

# --- Selecionar subconjunto pequeno para teste rápido ---
N = 100  # ajuste conforme necessário
subset_lbl = lbl_data[:N]
subset_tst = tst_data[:N]

# --- Gerar respostas usando model.generate() ---
generated_contents = []

for item in tqdm(subset_lbl, desc="Gerando respostas"):
    prompt = f"Pergunta sobre o título '{item['title']}': Descreva o produto."
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            do_sample=True,
            temperature=0.7
        )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Apenas o texto gerado após o prompt
    gen_response = generated_text[len(prompt):].strip()

    new_item = item.copy()
    new_item['content'] = gen_response
    generated_contents.append(new_item)

# --- Salvar o lbl.json preenchido ---
with open('/content/drive/MyDrive/Colab Notebooks/Datasets/lbl_generated.json', 'w') as f:
    for item in generated_contents:
        f.write(json.dumps(item) + '\n')

print("Arquivo lbl_generated.json salvo com as respostas geradas.")

# --- Avaliação: BLEU e ROUGE ---
bleu = load_metric("bleu")
rouge = load_metric("rouge")

# BLEU: listas de tokens
references = [[item['content'].split()] for item in subset_tst]
predictions = [item['content'].split() for item in generated_contents]

bleu_score = bleu.compute(predictions=predictions, references=references)
print(f"BLEU score: {bleu_score}")

# ROUGE: strings completas
references_str = [item['content'] for item in subset_tst]
predictions_str = [item['content'] for item in generated_contents]

rouge_score = rouge.compute(predictions=predictions_str, references=references_str)
print(f"ROUGE score: {rouge_score}")


TST samples: 970237, LBL samples: 1305265


Gerando respostas: 100%|██████████| 100/100 [01:34<00:00,  1.06it/s]


Arquivo lbl_generated.json salvo com as respostas geradas.
BLEU score: {'bleu': 0.0, 'precisions': [0.04956896551724138, 0.0014749262536873156, 0.0, 0.0], 'brevity_penalty': 0.1344630932475236, 'length_ratio': 0.3326164874551971, 'translation_length': 1392, 'reference_length': 4185}
ROUGE score: {'rouge1': AggregateScore(low=Score(precision=np.float64(0.0068237623246867754), recall=np.float64(0.004583458368861486), fmeasure=np.float64(0.003962582524801313)), mid=Score(precision=np.float64(0.016060562327446435), recall=np.float64(0.017171101000822364), fmeasure=np.float64(0.008676998078454751)), high=Score(precision=np.float64(0.028523152673824156), recall=np.float64(0.04299379618833178), fmeasure=np.float64(0.014552013309393283))), 'rouge2': AggregateScore(low=Score(precision=np.float64(0.0), recall=np.float64(0.0), fmeasure=np.float64(0.0)), mid=Score(precision=np.float64(0.0003448275862068965), recall=np.float64(0.00013245033112582781), fmeasure=np.float64(0.00019138755980861247)), h

1. Quantidade de dados:
TST samples: 970.237

LBL samples: 1.305.265

⚠️ Só foi gerado um subconjunto de 100 respostas para acelerar a execução.

2. Geração de textos:
O modelo gerou automaticamente descrições de produtos baseadas no título, salvando no arquivo lbl_generated.json.

3. Métricas de avaliação:
BLEU: mede quão próximas as respostas geradas estão das de referência.

bleu = 0.0: modelo não gerou textos próximos aos de referência.

Precisões: só a de 1-gramas tem valor (>1%), o resto é zero — indicando que sequências de várias palavras consecutivas não bateram.

Brevity penalty (0.31): penalidade porque os textos gerados foram muito mais curtos que os de referência.

length_ratio (0.46): textos gerados têm menos da metade do tamanho dos textos reais.

In [18]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

input_prompt = "What is Girls Ballet Tutu Neon?"

# Inferência
output_after = pipe(input_prompt, max_new_tokens=100)

print("Resposta DEPOIS do fine-tuning:\n", output_after[0]['generated_text'])


Device set to use cpu


Resposta DEPOIS do fine-tuning:
 What is Girls Ballet Tutu Neon?
Girls Ballet Tutu Neon is a product that is designed to provide a fun and colorful experience for girls. It is a tutu that is designed to be worn by girls and is made from a neon fabric. The tutu is designed to be comfortable and easy to wear, and it is perfect for ballet classes or other dance performances. The neon fabric is a fun and eye-catching color that will make any girl feel like a ballerina.


🚨 Fluxo resumido:
Antes do fine-tuning: cria pipeline com modelo base e gera texto.

Depois do fine-tuning: aplica LoRA, cria nova pipeline e gera texto.

Gera Perplexity que é uma medida de "confiança" do modelo em prever o texto. Quanto menor, melhor o modelo está em prever sequências de palavras.

Comparamos os resultados:



In [19]:
import torch
import math
from transformers import pipeline

def calculate_perplexity(model, tokenizer, input_text):
    model.eval()
    with torch.no_grad():
        inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
        outputs = model(**inputs, labels=inputs['input_ids'])
        loss = outputs.loss
        perplexity = math.exp(loss.item())
    return perplexity

# --- Prompt para teste ---
input_prompt = "What is Girls Ballet Tutu Neon?"

# --- Inferência qualitativa (geração de texto) ---

# Pipeline do modelo base (antes do fine-tuning)
pipe_before = pipeline("text-generation", model=base_model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)
output_before = pipe_before(input_prompt, max_new_tokens=100)
print("Resposta ANTES do fine-tuning:\n", output_before[0]['generated_text'])

# Pipeline do modelo fine-tuned (sem merge_and_unload)
pipe_after = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)
output_after = pipe_after(input_prompt, max_new_tokens=100)
print("\nResposta DEPOIS do fine-tuning:\n", output_after[0]['generated_text'])

# --- Comparação Quantitativa: Perplexity ---

# Perplexity antes do fine-tuning
ppl_before = calculate_perplexity(base_model, tokenizer, input_prompt)
print(f"\nPerplexity ANTES do fine-tuning: {ppl_before:.4f}")

# Perplexity depois do fine-tuning
ppl_after = calculate_perplexity(model, tokenizer, input_prompt)
print(f"Perplexity DEPOIS do fine-tuning: {ppl_after:.4f}")

# --- Avaliação ---
improvement = ppl_before - ppl_after
print(f"\nDiferença de Perplexity (antes - depois): {improvement:.4f}")


Device set to use cpu
Device set to use cpu


Resposta ANTES do fine-tuning:
 What is Girls Ballet Tutu Neon?
Girls Ballet Tutu Neon is a product that is designed to provide a fun and exciting way for girls to dance. The tutu is made from a soft and stretchy material that is perfect for dancing. The tutu is designed to be comfortable and easy to move in. The tutu is also designed to be lightweight, so that girls can move around easily and not feel weighed down.
The tutu is available in a variety of colors, including

Resposta DEPOIS do fine-tuning:
 What is Girls Ballet Tutu Neon?
Girls Ballet Tutu Neon is a product that is designed to provide a fun and colorful experience for girls. It is a tutu that is designed to be worn by girls and is made from a neon fabric. The tutu is designed to be comfortable and easy to wear, and it is perfect for ballet classes or other dance performances. The neon fabric is a fun and eye-catching color that will make any girl feel like a ballerina.

Perplexity ANTES do fine-tuning: 287.6886
Perplexity

Análise rápida do resultado:
Geração antes do fine-tuning: Texto coerente, mas mais genérico e repetitivo.

Geração depois do fine-tuning: Texto mais específico, com termos mais alinhados ao prompt ("colorful experience", "neon fabric"), indicando que o modelo aprendeu nuances do dataset.

Perplexity: Caiu de 287.7 para 284.2, uma melhora modesta, que indica o modelo está um pouco mais confiante na previsão dos tokens após o fine-tuning.

✅ Parte 5: Testando modelo para receber perguntas dos usuários.
O modelo gera uma resposta baseada na pergunta do usuário e nos dados provenientes do fine-tuning, incluindo as fontes fornecidas. .

In [3]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# --- Configurações ---

fine_tuned_model_path = "/content/ModeloTreinadoTechChallenge3/checkpoint-50"  # pasta do modelo fine-tuned
tokenizer_path = fine_tuned_model_path

tst_file = "/content/drive/MyDrive/Colab Notebooks/Datasets/tst.json"
output_file = "/content/drive/MyDrive/Colab Notebooks/Datasets/generated_answers.json"

# --- Carregar tokenizer e modelo fine-tuned ---

tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
model = AutoModelForCausalLM.from_pretrained(fine_tuned_model_path)
device = 0 if torch.cuda.is_available() else -1

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=device,
    max_new_tokens=100,
    do_sample=False
)

# --- Função para gerar resposta a partir da pergunta ---

def generate_answer(question_title):
    prompt = f"Pergunta sobre o título '{question_title}': Descreva o produto.\nResposta:"
    output = generator(prompt)
    generated_text = output[0]['generated_text']
    answer = generated_text[len(prompt):].strip()
    return answer

# --- Ler perguntas --- (limitado a 100)

questions = []
with open(tst_file, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        if i >= 100:
            break
        questions.append(json.loads(line))

# --- Gerar respostas ---

results = []

print(f"Gerando respostas para {len(questions)} perguntas...")

for idx, item in enumerate(questions, start=1):
    title = item.get("title")
    if not title:
        continue
    answer = generate_answer(title)
    results.append({
        "title": title,
        "generated_answer": answer
    })

    if idx % 100 == 0:
        print(f"{idx} respostas geradas...")

# --- Salvar respostas geradas ---

with open(output_file, "w", encoding="utf-8") as f:
    for r in results:
        f.write(json.dumps(r, ensure_ascii=False) + "\n")

print(f"Respostas geradas salvas em '{output_file}'.")

# --- Um exemplo para mostrar a estrutura ---

if results:
    print("\nExemplo de resposta gerada:")
    print(json.dumps(results[0], ensure_ascii=False, indent=2))


Device set to use cuda:0


Gerando respostas para 100 perguntas...
100 respostas geradas...
Respostas geradas salvas em '/content/drive/MyDrive/Colab Notebooks/Datasets/generated_answers.json'.

Exemplo de resposta gerada:
{
  "title": "Adult Ballet Tutu Cheetah Pink",
  "generated_answer": "This tutu is perfect for ballet performances, recitals, and other dance events. It is made of high-quality, breathable fabric that is perfect for dancing. The tutu is made of a soft, stretchy fabric that allows for easy movement and flexibility. The tutu is designed to fit most adults, and the cute cheetah print adds a fun touch to any performance. This tutu is perfect for any dance enthusiast, whether they are a"
}


✅ Considerações Finais

O fine-tuning usando LoRA mostrou ser eficiente para adaptar um modelo grande com poucos recursos.

A redução da perplexity indica uma melhora no desempenho do modelo em relação aos dados específicos.

A melhoria qualitativa nas respostas mostra que o modelo aprendeu a incorporar melhor as informações do dataset.

Para usos futuros, recomenda-se aumentar o número de épocas e expandir o dataset para maior robustez.

A abordagem modular (pré-processamento, treinamento, avaliação) facilita a manutenção e expansão do projeto.

O salvamento do modelo garante reprodutibilidade e facilidade de integração em aplicações práticas.